# Narupa Concepts: Selections & Visualisation

By creating selections of sets of atoms, we can display different atoms using different visualisations provided in the VR client Narupa iMD.

Selections are just like any other item stored in the multiplayer shared state - a dictionary of values with a given name. A selection is any object in the shared state which has a key of the form `selection.{id}`, such as `selection.1` or `selection.protein`. This id has to be unique, and hence you should not normally worry about it (it will be generated automatically)

## Start a Server

To begin to use selections, we will need a server! We will use the *Neuraminidase* example provided for the ASE/OpenMM runner, as it has a good mix of features including a polypeptide chain and multiple molecules (protein, ligand, and a couple of ions).

In [ ]:
from narupa.openmm import serializer

# Read the neuraminidase xml file, which defines the simulation parameters
with open('../ase/openmm_files/neuraminidase.xml') as infile:
    simulation = serializer.deserialize_simulation(infile.read())

In [ ]:
from narupa.ase.openmm import OpenMMIMDRunner

# Start a Narupa server given the simulation
server = OpenMMIMDRunner(simulation)

# Print what port the server is running on
print("Server running on port {0}".format(server.port))

# Launch the server
server.run()

We now have a server running on the given port. By starting up Narupa iMD, you can now connect to this server either by using the autoconnect feature, or by manually typing in the port and address (you can use either `localhost` or `127.0.0.1`, both of which represent your local machine)

Connecting to the server will present you with a simulation of Neuraminidase, drawn using the standard ball and stick approach. The coloring by default is **CPK**, which colors hydrogens, carbons, oxygens and nitrogens white, grey, red and blue respectively.

![Ball and Stick](images/ball-and-stick.png)

## Creating a Python Client

In order to make create and change selections, you require a client to be connected to the server. This client can then submit changes to the server's synchronised state, which is shared between everyone.

The Narupa iMD client does not currently have a way of altering selections, so we will be using a pythonic client here in the notebook. Imagine this client as a phantom player - they can get information about the server and make changes, but they don't appear in the virtual space as an avatar.

In [ ]:
from narupa.app.client import NarupaImdClient

# Start a Narupa client, using autoconnect to find the server we started earlier
client = NarupaImdClient.autoconnect()
client.subscribe_to_frames()

# Wait for the first frame to be recieved about the current state of the system
first_frame = client.wait_until_first_frame(check_interval=0.5, timeout=10)

print("Client connected, recieved frame with {0} atom(s)".format(first_frame.particle_count))

The client is now connected and recieving frames, however in order to be able to change the state it must join the multiplayer as well.

In [ ]:
# Begin tracking updates to the shared multiplayer values
client.subscribe_multiplayer()

# The Root Selection

Every simulation has an implicit selection already there - the **root selection**. This is an automatic selection that includes every atom in the simulation. We can access this in Python by accessing the `root_selection` property: 

In [ ]:
root_selection = client.root_selection
print(root_selection)

This gives us an object which we can use to manipulate the selection. It has several properties that we can inspect, such as:

* `selected_particle_ids` - The set of particle indices that are in this selection. A value of `None` (as is the case of the root selection) does **NOT** mean there are no particles in the selection - it means that it includes **ALL** particles. An **empty set** represents an empty selection. This convention is used at several points throughout Narupa iMD. The reasoning behind this is, it would be impractical for a system of 10,000 atoms to have to store a list `[0, 1, ..., 9998, 9999]` just represent a selection of everything.

In [ ]:
print(root_selection.selected_particle_ids)

* `selection_name` - As mentioned above, every selection has a name that will be displayed to the user.

In [ ]:
print(root_selection.selection_name)

* `selection_id` - A unique ID generated by your client to identify the selection. Note that the root selection has a unique ID `root` (and hence is stored in the shared state dictionary under the key `selection.root`). This is an important name, as it allows Narupa iMD to identify which selection should be used for particles who don't belong to any other.

In [ ]:
print(root_selection.selection_id)

* `interaction_method` - Specify what interaction method should be used with interacting with this selection. This can be
    * `single`: Standard interaction mode, interacting will grab the nearest atom
    * `group`: Grouped interaction mode, interacting with grab the entire selection
    * `none`: Disable interactions. If an atom is a member of this selection, it will not be interactable.

In [ ]:
print(root_selection.interaction_method)

<div class="alert alert-info">

**Note:** It is **strongly discouraged** to set the `interaction_method` for the root selection to `group`. This would mean if you interacted, you would interact with the entire simulation at once.

</div>

* `velocity_reset` - Setting this to true enabled velocity reset, which means when you release a particle after interacting with it, its velocities will be reset to 0. This prevents drifting when you are performing tasks such as drug docking

In [ ]:
print(root_selection.velocity_reset)

* `renderer` - This decides how the selection will be drawn. We will be going into more detail in the later section on visualisations.

In [ ]:
print(root_selection.renderer)

* `hide` - If set to True, this hides the selection from view. It overrides any renderer specified above, by stating that the selection should not be visualised in any way.

In [ ]:
print(root_selection.hide)

## Editing a Selection

As you can see above, we can access various properties of the root selection. We can also set these options, allowing us to make modifications which will be sent to all users.

However, simply setting a value **does not** send the change. To actually send the changes, you must use the `flush_changes` command. This schedules your changes to be sent to the server at the next possible opportunity, where they will then be sent to all other players.

Let's have an example. Let's disable interactions for our root selection:

In [ ]:
root_selection.interaction_method = 'none'
root_selection.flush_changes()

You should now note that you can no longer interact with the simulation. This is because currently, every atom belongs to the root selection and we have disabled interactions for it. Let's now try making multiple changes:

In [ ]:
root_selection.interaction_method = 'single'
root_selection.velocity_reset = True
root_selection.flush_changes()

It can be easy to forget to flush your changes after commands such as this. Therefore, we provide a syntax which automatically flushes changes:

In [ ]:
with root_selection.modify() as selection:
    selection.interaction_method = 'single'
    selection.velocity_reset = True

This is the prefered syntax for making changes to selections.

# Visualisations

As mentioned above, the `renderer` key of a selection determines the visualisation that will be used in Narupa iMD. By changing this, we can determine how we present our system to our users.

## Presets

The simplest way we can change our visualisation is to choose one of the preset visualisers provided in Narupa iMD. These are fully set up with parameters and colors to allow you to quickly choose an option. Anything possible with presets is also possible with the more detailed setup described below.

The possible presets are:

<table>
    <tr>
        <td style='text-align: center'>
            <code>"ball and stick"</code>
            <img src='./images/presets/ball-and-stick.png'>
        </td>
        <td style='text-align: center'>
            <code>"liquorice"</code>
            <img src='./images/presets/liquorice.png'>
        </td>
        <td style='text-align: center'>
            <code>"noodles"</code>
            <img src='./images/presets/noodles.png'>
        </td>
        <td style='text-align: center'>
            <code>"goodsell"</code>
            <img src='./images/presets/goodsell.png'>
        </td>
    </tr>
    <tr>
        <td style='text-align: center'>
            <code>"cycles"</code>
            <img src='./images/presets/cycles.png'>
        </td>
        <td style='text-align: center'>
            <code>"spline"</code>
            <img src='./images/presets/spline.png'>
        </td>
        <td style='text-align: center'>
            <code>"geometric spline"</code>
            <img src='./images/presets/geometric-spline.png'>
        </td>
        <td style='text-align: center'>
            <code>"cartoon"</code>
            <img src='./images/presets/cartoon.png'>
        </td>
    </tr>
</table>

We choose a visualiser in the same way we alter any other properties of a selection:

In [ ]:
# Set the root selection to be drawn using the liquorice preset
with client.root_selection.modify() as selection:
    selection.renderer = 'liquorice'

## Constructing a Visualiser

If we want to get our hands dirty and have more control over how our system is displayed, we can construct a visualiser. This involves setting the renderer to a dictionary with specific keys, which are instructions on how the client should build a visualiser.

A visualiser consists of several parts linked together, which gives a modular construction that allows more control how different values are calculated:

* A **sequence** component (Optional), which calculates sequences of particles indices for drawing as splines.
* A **color** component, which provides a per particle color scheme
* A **scale** component, which provides a per particle scale
* A **width** component, which provides a per particle width (used for visualisers such as ribbons)
* A **render** component, who takes information from the other components and draws something to the screen

All of these parts have a default value, so technically an empty dictionary is a valid renderer:

In [ ]:
# Set the root selection to be drawn using the liquorice preset
with client.root_selection.modify() as selection:
    selection.renderer = {}

You will see the following visualisation appear:

![](./images/examples/empty-dict.png)

This is because the defaults are as follows
* If you don't provide a color, white is used
* If you don't provide a scale, it is 1
* If you don't provide a render, it defaults to ball and stick

Let's try something a little more interesting. Let's pick a color such as cornflower blue, and try it out

In [ ]:
# Set the root selection to be drawn using the liquorice preset
with client.root_selection.modify() as selection:
    selection.renderer = {
        "color": "CornflowerBlue",
        "render": "liquorice"
    }

![](./images/examples/cornflower-blue.png)

<div class="alert alert-info">

**Colors:** Colors can be specified in several different ways:
    
   * A string containing a hex color, with or without a preceding hash, such as `'F15H3C'` or `'#FF4E00'`. Googling 'color picker' will give you a widget that allows you to choose a color.
   * Any of the colors defined in the CSS3 standard, such as `'Red'` or `'AliceBlue'`. These will be familiar with those involved in web development, with a full list available [here](https://www.w3schools.com/cssref/css_colors.asp)
   * An array of three floats, defining the R, G and B values from 0 to 1, such as `'[0.3, 0.8, 1.0]'`

Any variable which accepts a color can be set using any of these methods

</div>

For certain arguments such as `color`, `render` and `scale`, providing the name of a specific component (in the above case, we are using the `liquorice` render component) will use that component in the visualiser. A **component** is a part of the visualiser that takes in some data, and outputs some new data. For example, the `cpk` color component takes in the element of each particle, and outputs a color based upon a defined scheme. 

Each component may also has optional arguments which can be modified. Here, instead of just giving the string name of the component, we must provide a dictionary which has a 'type': 'component_name' entry. For example, the `cpk` color component has a parameter `scheme`:

In [ ]:
# Use the CPK component with default arguments
with client.root_selection.modify() as selection:
    selection.renderer = {
        "color": "cpk"
    }

![](./images/examples/cpk.png)

In [ ]:
# Use the CPK component with the 'scheme' argument, which specifies the built-in Narupa Scheme
with client.root_selection.modify() as selection:
    selection.renderer = {
        "color": {
            'type': 'cpk',
            'scheme': 'narupa'
        }
    }

![](./images/examples/cpk-narupa.png)

In [ ]:
# Use the CPK component with the 'scheme' argument, but providing your own scheme
with client.root_selection.modify() as selection:
    selection.renderer = {
        "color": {
            'type': 'cpk',
            'scheme': {
                'H': 'HoneyDew',
                'C': 'DarkSlateGrey',
                'O': 'FireBrick',
                'N': 'DarkTurquoise',
                'S': 'Khaki'
            }
        }
    }

![](./images/examples/cpk-custom.png)

Once we have a scheme we quite like, we could also play around at modifying the scales of the ball and sticks:

In [ ]:
# Use the CPK component with the 'scheme' argument, but providing your own scheme and customising the render
with client.root_selection.modify() as selection:
    selection.renderer = {
        "color": {
            'type': 'cpk',
            'scheme': {
                'H': 'HoneyDew',
                'C': 'DarkSlateGrey',
                'O': 'FireBrick',
                'N': 'DarkTurquoise',
                'S': 'Khaki'
            }
        },
        "render": {
            "particle.scale": 0.05,
            "bond.scale": 0.02,
            "type": "ball and stick"
        }
    }

![](./images/examples/cpk-custom2.png)

The Appendix lists all current possible visualisations and their arguments that can be used in Narupa iMD.

# Selections

In preparation for this section, let's set our base visualiser to something clean:

In [ ]:
with client.root_selection.modify() as selection:
    selection.renderer = {
        'color': {
            'type': 'cpk',
            'scheme': 'narupa'
        },
        'scale': 0.04,
        'render': 'liquorice'
    }

![](./images/examples/clean.png)

The `NaruaImdClient` python client has several methods related to creating, updating and removing selections. First of all, we will be creating a selection. To do so, we need two things

* A name. This is currently unused, however it may be displayed in the future in a user interace, so choose a concise descriptive name of the selection you are creating

* A list of particle indices that will be in your selection. 

<div class="alert alert-info">

**Note:** Particle indices are **0-based**, as defined in your simulation. Therefore, the first particle is **0**, the second particle is **1**, etc.

</div>

For our example here, we will be creating a selection consisting of ARG-37 and GLU-38, which consists of all particles from 563 to 601:

In [ ]:
my_first_selection = client.create_selection("ARG_GLU", list(range(563, 602)))

Note that we could have just as easily written the particle indices out as `[563, 564, ..., 600, 601]`, however we are using the `range` command for brevity. Also remember that the range command is not end inclusive - `602` does not appear in the list.

You may not notice the selection initially. This is because the selection has defaulted to the default renderer. Let's highlight this selection instead:

In [ ]:
# Draw the selection in a chunky gold liquorice style
with my_first_selection.modify() as selection:
    selection.renderer = {
        'color': 'Gold',
        'scale': 0.1,
        'render': 'liquorice'
    }

![](./images/examples/selection1.png)

We can now see our selection! All the atoms that we specified in this selection are now being draw using the renderer specified for our new selection instead.

After creating our selection, we can easily expand it in two different ways - we can set the particles using `set_particles`, or expand our selection using `add_particles`:

In [ ]:
# Change selection to be ARG-144 and THR-145
with my_first_selection.modify() as selection:
    selection.set_particles(range(2213, 2250))

In [ ]:
# Expand the above selection to include LEU-143
with my_first_selection.modify() as selection:
    selection.add_particles(range(2194, 2213))

We have now changed our selection to be LEU-143, ARG-144 and THR-145

![](./images/examples/selection2.png)

If we ever want to get rid of our selection, we can do so using the `remove` method. Note this is the only method where you **do not** need to call `modify()` or `flush_changes()` - calling remove instantly sends a message to the server to remove this selection from everyone's dictionaries

In [ ]:
# Remove our selection
my_first_selection.remove()

If we have several selections and we want to clear them all, our client provides a handy method for this

In [ ]:
# Remove all selections
client.clear_selections()

## Close the Server

It is good practise to close the client and server when we are done, to clean up any resources and ensure they're not still running in the background.

In [ ]:
client.close()
server.close()

# Next Steps

* Try out visualisations as a part of a tutorial, such as static snapshot of an [seratonin receptor](../mdanalysis/mdanalysis_lsd.ipynb) or setting up the [neuraminidase/oesteltamivir](../ase/openmm_neuraminidase.ipynb) simulation fro scratch!

# Appendix - Possible Options for Visualisers

## Parameter Types

Parameters can be of the following types, and can often be specified in different ways:

#### Float

Can be specified as either an integer (`0`, `32`, `-11`) or a decimal (`2.31`, `-0.2`).

#### String

Self explanatory, use a python string

#### Color

Explained in more detail under the Visualisation section, but can be

* A string containing a hex code, with or without preceding #
* A string specifying a CSS3 color name
* A list of three floats, specifying the R, G and B values from 0 to 1

#### Gradient

A gradient is a list of two to seven colors, each specified as above. You can mix and match which styles you use

```python
[
    '#ffff00',
    'SlateGrey',
    [0.1, 0.5, 0.3]
]
```

#### Element-Color Mapping

Defines a map between atomic elements and colors. It can be

* A predefined map. The current options are `jmol` or `narupa`.
* A custom dictionary. The keys of this should be atomic elements, and colors can be specified as above
```python
{
    'H': 'AliceBlue',
    'O': '#ff2e09'
}
```

---

## Color Components

The following components can be used by adding `'color': component_name` to your dictionary. For parameters, you should using

```python
'color': {
    'type': component_name,
    # parameters...
}
```

If you want a solid color, you can simply specify `'color': color` using the color specification defined above

## `cpk`

Color by atomic element

**Parameters**

* `scheme` (Element-Color Mapping) - The mapping to use to transform element to color. Defaults to `jmol`.

## `goodsell`

Goodsell style color scheme. Cycles through a set of 11 pastel colors, coloring each entity uniquely. Colors non carbons in a slightly darker color

## `particle index`

Colors particles by their index in the simulation, with the gradient going from the first particle to the last particle.

**Parameters**

* `gradient` (Gradient) - The gradient to use to color the particles Defaults to a red to blue gradient.

## `residue index`

Colors particles by the index of their residue in the simulation, with the gradient going from the first residue to the last.

**Parameters**

* `gradient` (Gradient) - The gradient to use to color the particles Defaults to a red to blue gradient.

## `residue index in entity`

Colors particles by the index of their residue in their entity, with the gradient going from the first residue of each entity to the last.

**Parameters**

* `gradient` (Gradient) - The gradient to use to color the particles Defaults to a red to blue gradient.

## `residue name`

Colors particles by the name of their residue. Currently hardcoded to use the RCSB colors, but will be modifiable in a future update.

## `secondary structure`

Colors particles by their secondary structure, calculated using the DSSP algorithm. Currently the color is hard coded to:

* Yellow for beta sheets
* Red for pi helices
* Magenta for alpha helices
* Blue for 3-10 helices
* White for everything else

These colors will be modifiable in a future update.

---

## Scale Components

The following components can be used by adding `'scale': component_name` to your dictionary. For parameters, you should using

```python
'scale': {
    'type': component_name,
    # parameters...
}
```

If you want a constant scale, you can simply specify `'scale': scale` using the float specification defined above

## `vdw`

Use the van der Waals radii for each element.

---

## Width Components

The following components can be used by adding `'width': component_name` to your dictionary. For parameters, you should using

```python
'width': {
    'type': component_name,
    # parameters...
}
```

If you want a constant width, you can simply specify `'width': width` using the float specification defined above

## `secondary structure`

Use the following widths based on the calculated secondary structure

* 1 for helices and sheets
* 0 for everything else

This gives width to sheets and helices to allow a ribbon style render

---

## Sequence Components

The following components can be used by adding `'sequence': component_name` to your dictionary. For parameters, you should using

```python
'sequence': {
    'type': component_name,
    # parameters...
}
```

You need to provide a sequence component when dealing with render components such as splines, which you need to tell how to decide which curves to draw

## `entities`

Create sequences that go from the first particle in each entity to the last.

## `polypeptide`

Create sequences of the alpha carbons of subsequent amino acids. All non amino acids are ignored.

---

## Render Components

Render components are responsible for drawing objects to the screen.

The following components can be used by adding `'render': component_name` to your dictionary. For parameters, you should using

```python
'render': {
    'type': component_name,
    # parameters...
}
```

## `ball and stick`

Classic ball and stick render

**Parameters**

* `color` (Color) - Color which is multiplied to whatever is provided from before. Default is white.
* `particle.scale` (Float) - The scaling factor of the balls. Default is 0.1x
* `bond.scale` (Float) - The scaling factor of the bonds. Default is 0.05x

## `liquorice`

Liquorice render, equivelant to a ball and stick render where the radii of the balls and sticks are equal

**Parameters**

* `color` (Color) - Color which is multiplied to whatever is provided from before. Default is white.
* `scale` (Float) - Scaling factor. Default is 0.05x

## `cycles`

Similar to a liquorice render, but with any rings with between three and six members shaded using a convex hull.

**Parameters**

* `color` (Color) - Color which is multiplied to whatever is provided from before. Default is white.
* `scale` (Float) - Scaling factor. Default is 0.1x

## `goodsell`

Renders standard spheres, but without shading. This also applies an outline based upon the particle's residue, helping to highlight the chains in the molecule.

**Parameters**

* `color` (Color) - Color which is multiplied to whatever is provided from before. Default is white.
* `scale` (Float) - Scaling factor. Default is 0.6x

## `noodles`

Similar to the liquorice render, but curving bonds to give a more organic shape.

**Parameters**

* `color` (Color) - Color which is multiplied to whatever is provided from before. Default is white.
* `scale` (Float) - Scaling factor. Default is 0.1x

## `spline`

Draws a cylinder along each sequence of particles.

**Parameters**

* `scale` (Float) - Scaling factor. Default is 0.1x

## `geometric spline`

Uses a method similar to the cycles renderer to give a chunky geometric representation of a ribbon along each sequence of particles.

**Parameters**

* `scale` (Float) - Scaling factor, describing how chunky the spline is. Default is 0.1x
* `width` (Float) - Scaling factor, describing the width of the spline. Default is 0.1

## `elliptic spline`

Draw a cylindrical spline which can be stretched into an elliptic spline, with a fix to ensure that glitching does not occur when the spline rotates.

**Parameters**

* `scale` (Float) - Scaling factor, describing how chunky the spline is. Default is 0.1x
* `width` (Float) - Scaling factor, describing the width of the spline. Default is 0.1